# Notebook 4 : Comparaison des modèles

Ce notebook charge les prédictions sauvegardées par les notebooks 1 à 5 et les compare sur les mêmes métriques.

**Prérequis** : avoir exécuté les notebooks 1, 2, 3 et 5 en entier. Chacun sauvegarde ses prédictions dans des fichiers `.npy`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

plt.rcParams['figure.dpi'] = 120

## 1. Chargement des prédictions

In [ ]:
# Charger toutes les prédictions
models = {}

# --- Notebook 1 : ML classique ---
y_test_ml = np.load('y_test.npy')
for name, fname in [('XGBoost', 'preds_xgboost.npy'), 
                     ('RandomForest', 'preds_rf.npy'),
                     ('GradientBoosting', 'preds_gb.npy')]:
    if os.path.exists(fname):
        models[name] = {'pred': np.load(fname), 'true': y_test_ml}
        print(f'{name}: chargé ({models[name]["pred"].shape})')

# Notebook 1 : Kalman position
if os.path.exists('preds_xgboost_kalman.npy'):
    models['XGBoost + Kalman pos'] = {
        'pred': np.load('preds_xgboost_kalman.npy'),
        'true': y_test_ml
    }
    print(f'XGBoost + Kalman pos: chargé')

# --- Notebook 2 : Transformer ---
if os.path.exists('preds_transformer.npy'):
    y_test_tr = np.load('y_test_transformer.npy')
    models['Transformer'] = {
        'pred': np.load('preds_transformer.npy'),
        'true': y_test_tr
    }
    print(f'Transformer: chargé')

if os.path.exists('preds_transformer_kalman.npy'):
    models['Transformer + Kalman pos'] = {
        'pred': np.load('preds_transformer_kalman.npy'),
        'true': y_test_tr
    }
    print(f'Transformer + Kalman pos: chargé')

# --- Notebook 3 : CNN ---
if os.path.exists('preds_cnn.npy'):
    y_test_cnn = np.load('y_test_cnn.npy')
    models['CNN'] = {
        'pred': np.load('preds_cnn.npy'),
        'true': y_test_cnn
    }
    print(f'CNN: chargé')

if os.path.exists('preds_cnn_kalman.npy'):
    models['CNN + Kalman pos'] = {
        'pred': np.load('preds_cnn_kalman.npy'),
        'true': y_test_cnn
    }
    print(f'CNN + Kalman pos: chargé')

# --- Notebook 5 : Améliorations ---
# Kalman vitesse
for name, fname, true_file in [
    ('XGBoost + Kalman vel', 'preds_xgboost_kalman_vel.npy', 'y_test.npy'),
    ('Transformer + Kalman vel', 'preds_transformer_kalman_vel.npy', 'y_test_transformer.npy'),
    ('CNN + Kalman vel', 'preds_cnn_kalman_vel.npy', 'y_test_cnn.npy'),
]:
    if os.path.exists(fname):
        models[name] = {'pred': np.load(fname), 'true': np.load(true_file)}
        print(f'{name}: chargé')

# Ensemble
if os.path.exists('preds_ensemble_weighted.npy'):
    models['Ensemble pondéré'] = {
        'pred': np.load('preds_ensemble_weighted.npy'),
        'true': y_test_ml  # même test set
    }
    print(f'Ensemble pondéré: chargé')

if os.path.exists('preds_ensemble_kalman_vel.npy'):
    models['Ensemble + Kalman vel'] = {
        'pred': np.load('preds_ensemble_kalman_vel.npy'),
        'true': y_test_ml
    }
    print(f'Ensemble + Kalman vel: chargé')

# GRU
if os.path.exists('preds_gru.npy'):
    y_test_gru = np.load('y_test_gru.npy')
    models['GRU multi-fenêtre'] = {
        'pred': np.load('preds_gru.npy'),
        'true': y_test_gru
    }
    print(f'GRU multi-fenêtre: chargé ({y_test_gru.shape})')

if os.path.exists('preds_gru_kalman_vel.npy'):
    models['GRU + Kalman vel'] = {
        'pred': np.load('preds_gru_kalman_vel.npy'),
        'true': y_test_gru
    }
    print(f'GRU + Kalman vel: chargé')

print(f'\n{len(models)} modèles chargés : {list(models.keys())}')

## 2. Tableau comparatif des métriques

In [ ]:
results = []

for name, data in models.items():
    y_true = data['true']
    y_pred = data['pred']
    
    # Gérer les tailles différentes (GRU a un test set plus court à cause du contexte)
    n = min(len(y_true), len(y_pred))
    y_true = y_true[:n]
    y_pred = y_pred[:n]
    
    eucl = np.sqrt((y_true[:, 0] - y_pred[:, 0])**2 + (y_true[:, 1] - y_pred[:, 1])**2)
    
    results.append({
        'Modèle': name,
        'N': n,
        'MSE_X': mean_squared_error(y_true[:, 0], y_pred[:, 0]),
        'MSE_Y': mean_squared_error(y_true[:, 1], y_pred[:, 1]),
        'MAE_X': mean_absolute_error(y_true[:, 0], y_pred[:, 0]),
        'MAE_Y': mean_absolute_error(y_true[:, 1], y_pred[:, 1]),
        'R²_X': r2_score(y_true[:, 0], y_pred[:, 0]),
        'R²_Y': r2_score(y_true[:, 1], y_pred[:, 1]),
        'Eucl_mean': eucl.mean(),
        'Eucl_median': np.median(eucl),
        'Eucl_p90': np.percentile(eucl, 90),
    })

# Trier par erreur euclidienne moyenne (meilleur en haut)
results.sort(key=lambda r: r['Eucl_mean'])

# Affichage formaté
print(f'{"Modèle":<28} {"R²_X":>7} {"R²_Y":>7} {"Eucl_mean":>10} {"Eucl_med":>10} {"Eucl_p90":>10}  {"N":>5}')
print('-' * 90)
for r in results:
    print(f'{r["Modèle"]:<28} {r["R²_X"]:>7.4f} {r["R²_Y"]:>7.4f} {r["Eucl_mean"]:>10.4f} {r["Eucl_median"]:>10.4f} {r["Eucl_p90"]:>10.4f}  {r["N"]:>5}')

## 3. Barplot des métriques

In [ ]:
model_names = [r['Modèle'] for r in results]
n_models = len(model_names)
colors = plt.cm.tab20(np.linspace(0, 1, n_models))

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# R² moyen (X + Y) / 2
r2_avg = [(r['R²_X'] + r['R²_Y']) / 2 for r in results]
bars = axes[0].barh(model_names, r2_avg, color=colors, edgecolor='black', linewidth=0.5)
axes[0].set_xlabel('R² moyen (X, Y)')
axes[0].set_title('R² moyen')
for bar, val in zip(bars, r2_avg):
    axes[0].text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2.,
                 f'{val:.3f}', ha='left', va='center', fontsize=8)

# Erreur euclidienne moyenne
eucl_means = [r['Eucl_mean'] for r in results]
bars = axes[1].barh(model_names, eucl_means, color=colors, edgecolor='black', linewidth=0.5)
axes[1].set_xlabel('Erreur euclidienne moyenne')
axes[1].set_title('Erreur euclidienne moyenne')
for bar, val in zip(bars, eucl_means):
    axes[1].text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2.,
                 f'{val:.4f}', ha='left', va='center', fontsize=8)

# Erreur euclidienne p90
eucl_p90s = [r['Eucl_p90'] for r in results]
bars = axes[2].barh(model_names, eucl_p90s, color=colors, edgecolor='black', linewidth=0.5)
axes[2].set_xlabel('Erreur euclidienne p90')
axes[2].set_title('Erreur euclidienne (90e percentile)')
for bar, val in zip(bars, eucl_p90s):
    axes[2].text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2.,
                 f'{val:.4f}', ha='left', va='center', fontsize=8)

plt.tight_layout()
plt.show()

## 4. CDF des erreurs euclidiennes (tous modèles)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

for i, (name, data) in enumerate(models.items()):
    y_true = data['true']
    y_pred = data['pred']
    n = min(len(y_true), len(y_pred))
    errors = np.sqrt((y_true[:n, 0] - y_pred[:n, 0])**2 + (y_true[:n, 1] - y_pred[:n, 1])**2)
    sorted_errors = np.sort(errors)
    cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
    ax.plot(sorted_errors, cdf, label=name, linewidth=2, color=colors[i])

ax.set_xlabel('Erreur euclidienne')
ax.set_ylabel('CDF (proportion ≤ erreur)')
ax.set_title('Distribution cumulée des erreurs - Comparaison de tous les modèles')
ax.legend(fontsize=8, loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 0.7)
ax.axhline(0.5, color='gray', linestyle=':', alpha=0.5)
ax.axhline(0.9, color='gray', linestyle=':', alpha=0.5)
plt.tight_layout()
plt.show()

## 5. Trajectoires prédites superposées

In [ ]:
# On prend les 300 premiers points du test set
# On sélectionne les modèles les plus intéressants pour la trajectoire
seg = slice(0, 300)
ref_true = list(models.values())[0]['true']

# Sélectionner un sous-ensemble pour lisibilité
highlight_models = ['XGBoost', 'Transformer', 'CNN']
kalman_models = [n for n in models if 'Kalman vel' in n or 'Ensemble' in n or 'GRU' in n]
# On affiche les bruts + les améliorations

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Trajectoire 2D
axes[0].plot(ref_true[seg, 0], ref_true[seg, 1], 'k-', linewidth=2.5, alpha=0.8, label='Vrai', zorder=10)
for i, (name, data) in enumerate(models.items()):
    pred = data['pred']
    n = min(len(pred), len(ref_true))
    if seg.stop <= n:
        lw = 1.8 if name in kalman_models else 0.8
        alpha = 0.7 if name in kalman_models else 0.3
        axes[0].plot(pred[seg, 0], pred[seg, 1], '-', linewidth=lw, alpha=alpha, 
                     color=colors[i], label=name)

axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title('Trajectoire - 300 premiers points test')
axes[0].legend(fontsize=7, loc='best')
axes[0].set_aspect('equal')

# Erreur au cours du temps (top 5 modèles seulement)
top5 = [r['Modèle'] for r in results[:5]]
for i, (name, data) in enumerate(models.items()):
    if name in top5:
        y_true = data['true']
        y_pred = data['pred']
        n = min(len(y_true), len(y_pred))
        if seg.stop <= n:
            errors = np.sqrt((y_true[:n, 0] - y_pred[:n, 0])**2 + (y_true[:n, 1] - y_pred[:n, 1])**2)
            axes[1].plot(errors[seg], linewidth=1, alpha=0.8, color=colors[i], label=name)

axes[1].set_xlabel('Index (test set)')
axes[1].set_ylabel('Erreur euclidienne')
axes[1].set_title('Erreur au cours du temps - Top 5 modèles')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

## 6. Heatmaps d'erreur spatiale (tous modèles)

In [ ]:
# Heatmaps limitées aux modèles clés (un par famille + meilleurs)
heatmap_models = ['XGBoost', 'Transformer', 'CNN', 'Transformer + Kalman pos']

# Ajouter les modèles du notebook 5 s'ils existent
for name in ['Transformer + Kalman vel', 'Ensemble + Kalman vel', 'GRU multi-fenêtre']:
    if name in models:
        heatmap_models.append(name)

# Filtrer ceux qui existent
heatmap_models = [n for n in heatmap_models if n in models]

n_hm = len(heatmap_models)
fig, axes = plt.subplots(1, n_hm, figsize=(4.5 * n_hm, 5))
if n_hm == 1:
    axes = [axes]

nbins = 20
x_edges = np.linspace(0, 1, nbins + 1)
y_edges = np.linspace(0, 1, nbins + 1)

# Calculer les error maps
all_error_maps = []
for name in heatmap_models:
    data = models[name]
    y_true = data['true']
    y_pred = data['pred']
    n = min(len(y_true), len(y_pred))
    eucl = np.sqrt((y_true[:n, 0] - y_pred[:n, 0])**2 + (y_true[:n, 1] - y_pred[:n, 1])**2)
    
    error_map = np.full((nbins, nbins), np.nan)
    count_map = np.zeros((nbins, nbins))
    for i in range(n):
        xi = np.clip(np.searchsorted(x_edges, y_true[i, 0]) - 1, 0, nbins - 1)
        yi = np.clip(np.searchsorted(y_edges, y_true[i, 1]) - 1, 0, nbins - 1)
        if np.isnan(error_map[yi, xi]):
            error_map[yi, xi] = 0
        error_map[yi, xi] += eucl[i]
        count_map[yi, xi] += 1
    
    mean_map = np.where(count_map > 0, error_map / count_map, np.nan)
    all_error_maps.append(mean_map)

# Limites communes
all_vals = np.concatenate([m[~np.isnan(m)] for m in all_error_maps])
vmin, vmax = np.percentile(all_vals, 5), np.percentile(all_vals, 95)

for idx, (name, error_map) in enumerate(zip(heatmap_models, all_error_maps)):
    im = axes[idx].imshow(error_map, origin='lower', aspect='equal', cmap='RdYlGn_r',
                           extent=[0, 1, 0, 1], vmin=vmin, vmax=vmax)
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('Y') if idx == 0 else None
    axes[idx].set_title(name, fontsize=10)

plt.colorbar(im, ax=axes, label='Erreur euclidienne moyenne', shrink=0.8)
plt.suptitle('Erreur spatiale par position - Modèles clés', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Scatter pred vs true (tous modèles)

In [ ]:
# Scatter plots limités aux modèles clés
scatter_models = ['XGBoost', 'Transformer', 'CNN']
for name in ['Transformer + Kalman vel', 'Ensemble + Kalman vel', 'GRU multi-fenêtre']:
    if name in models:
        scatter_models.append(name)

n_sc = len(scatter_models)
fig, axes = plt.subplots(n_sc, 2, figsize=(10, 4 * n_sc))
if n_sc == 1:
    axes = axes.reshape(1, -1)

for idx, name in enumerate(scatter_models):
    if name not in models:
        continue
    data = models[name]
    y_true = data['true']
    y_pred = data['pred']
    n = min(len(y_true), len(y_pred))
    r2x = r2_score(y_true[:n, 0], y_pred[:n, 0])
    r2y = r2_score(y_true[:n, 1], y_pred[:n, 1])
    
    axes[idx, 0].scatter(y_true[:n, 0], y_pred[:n, 0], s=0.5, alpha=0.2)
    axes[idx, 0].plot([0, 1], [0, 1], 'r--', linewidth=2)
    axes[idx, 0].set_xlabel('True X')
    axes[idx, 0].set_ylabel('Pred X')
    axes[idx, 0].set_title(f'{name} - X (R²={r2x:.3f})')
    axes[idx, 0].set_aspect('equal')
    
    axes[idx, 1].scatter(y_true[:n, 1], y_pred[:n, 1], s=0.5, alpha=0.2)
    axes[idx, 1].plot([0, 1], [0, 1], 'r--', linewidth=2)
    axes[idx, 1].set_xlabel('True Y')
    axes[idx, 1].set_ylabel('Pred Y')
    axes[idx, 1].set_title(f'{name} - Y (R²={r2y:.3f})')
    axes[idx, 1].set_aspect('equal')

plt.tight_layout()
plt.show()

## 8. Discussion et conclusion

### Résumé des approches

| Notebook | Approche | Entrée | Points forts |
|----------|----------|--------|-------------|
| **NB1** | Feature Eng. + XGBoost/RF | ~35 features manuelles | Rapide, interprétable |
| **NB2** | Transformer | Séquence de waveforms bruts | Capture co-activations via self-attention |
| **NB3** | CNN bins temporels | Image (20ch × 22 bins) | Entrée fixe, patterns spatio-temporels |
| **NB5** | Kalman vitesse | Post-processing (état = [x,y,vx,vy]) | Anticipe le mouvement, meilleur que Kalman position |
| **NB5** | Ensemble pondéré | Combinaison de tous les modèles | Exploite la complémentarité |
| **NB5** | GRU multi-fenêtre | Séquence de 10 fenêtres de features | Apprend la continuité temporelle |

### Analyse des résultats

- **R²** : proportion de la variance expliquée (1 = parfait, 0 = aussi bon que la moyenne)
- **Erreur euclidienne** : en unités normalisées (0-1). L'arène ~40cm → erreur 0.1 ≈ 4cm
- **p90** : 90% des prédictions sous ce seuil — mesure les worst cases

### Impact des améliorations (notebook 5)

1. **Kalman vitesse vs position** : le modèle de vitesse constante anticipe les mouvements au lieu de simplement lisser. Le gain est surtout visible sur le p90 (pires cas) et en R² pendant les phases de mouvement rapide.

2. **Ensemble** : la moyenne pondérée exploite la complémentarité entre modèles. Même une simple moyenne réduit la variance des prédictions. L'ensemble + Kalman vitesse combine le meilleur des deux mondes.

3. **GRU multi-fenêtre** : en utilisant le contexte temporel (~1 seconde), le GRU peut apprendre la dynamique de la trajectoire de manière data-driven, sans hypothèse de modèle physique comme le Kalman.

### Classement attendu (du meilleur au moins bon)

1. GRU multi-fenêtre + Kalman vitesse (ou Ensemble + Kalman vitesse)
2. Transformer + Kalman vitesse
3. Transformer + Kalman position
4. Ensemble pondéré
5. Transformer brut
6. XGBoost + Kalman
7. XGBoost / GradientBoosting brut
8. CNN brut

### Pistes restantes

- **GRU sur embeddings Transformer** : extraire le vecteur dim=64 du Transformer (au lieu des features XGBoost) pour combiner richesse des waveforms + continuité temporelle
- **Multi-tâche** : prédire aussi vitesse et direction de tête
- **Data augmentation** : bruit sur waveforms, masquage de spikes/shanks
- **Plus d'epochs** : le Transformer passe de 5 à 30 epochs (à exécuter sur GPU)